#### Name: Noha Magdy Ali 
#### ID: 1610126
# Age and Gender Recognition using openCV and CNNs 

### Needed packages 

In [16]:
from skimage import data, io
from matplotlib import pyplot as plt
import numpy as np
from skimage import img_as_ubyte,img_as_float
from skimage import data, io, filters
from matplotlib.pyplot import imshow, show
from skimage.morphology import watershed
from skimage.measure import label
from skimage.color import label2rgb
from skimage import feature
from skimage.filters import threshold_otsu,  threshold_local, sobel
from skimage.morphology import erosion,dilation,opening,closing,square,disk
from skimage.measure import label
from skimage.color import label2rgb
import os
import pandas as pd
!pip install -U tf-nightly
import tensorflow as tf
import functools
!pip install mitdeeplearning
import mitdeeplearning as mdl
from tqdm import tqdm
import cv2
from skimage.transform import rescale, resize
from skimage.color import rgb2gray
print(tf.__version__)



Requirement already up-to-date: tf-nightly in /usr/local/lib/python3.6/dist-packages (2.3.0.dev20200529)
2.3.0-dev20200529


### Loading and Preparing Data

In [0]:
# file_id = '0BxYys69jI14kSVdWWllDMWhnN2c'
# request = drive_service.files().get_media(fileId=file_id)
# fh = io.BytesIO()
# downloader = MediaIoBaseDownload(fh, request)
# done = False
# while done is False:
#     status, done = downloader.next_chunk()
#     print ("Download %d%%." % int(status.progress() * 100))

# !wget -P /usr https://drive.google.com/drive/u/0/folders/0BxYys69jI14kSVdWWllDMWhnN2c

# my_tar = tarfile.open("/usr/0BxYys69jI14kSVdWWllDMWhnN2c.tar.gz")
# my_tar.extractall('/usr/Dataset') # specify which folder to extract to
# my_tar.close()




In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
path= "/content/drive/My Drive/Dataset/"
data= os.listdir(path)
#print (data)
Data = pd.DataFrame(columns=["Image", "Age", "Gender"])


for idx,i in zip(range(len(data)),data):
    age, gender, race, name=i.split("_")
    img = io.imread(path+i)
    img = resize(img, (512, 512))
    img = rgb2gray(img)
    img=img_as_ubyte(img)
    img=np.array(img)
    Data.loc[idx] = [ img, age, gender]
    #append(pd.Series(data={"Image":img, "Age":age, "Gender":gender}, name=idx))

In [36]:
Data.shape

(3251, 3)

In [37]:
Data.head()

,Image,Age,Gender
0,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",61,0
1,"[[153, 174, 196, 206, 207, 205, 206, 206, 206,...",40,0
2,"[[253, 253, 253, 252, 252, 252, 251, 252, 252,...",45,0
3,"[[30, 27, 47, 67, 90, 105, 95, 86, 88, 91, 91,...",41,0
4,"[[177, 177, 177, 178, 178, 178, 179, 179, 181,...",54,0


In [38]:
rand_idxs = np.random.choice(Data.shape[0], 500, True)
test = Data.loc[rand_idxs]
train = Data
train.drop(rand_idxs)

,Image,Age,Gender
0,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",61,0
1,"[[153, 174, 196, 206, 207, 205, 206, 206, 206,...",40,0
2,"[[253, 253, 253, 252, 252, 252, 251, 252, 252,...",45,0
3,"[[30, 27, 47, 67, 90, 105, 95, 86, 88, 91, 91,...",41,0
4,"[[177, 177, 177, 178, 178, 178, 179, 179, 181,...",54,0
...,...,...,...
3244,"[[76, 76, 76, 76, 76, 76, 76, 76, 76, 76, 76, ...",68,0
3245,"[[180, 180, 180, 180, 180, 180, 180, 180, 180,...",70,0
3246,"[[152, 152, 152, 152, 152, 152, 152, 152, 152,...",65,0
3247,"[[127, 127, 127, 127, 127, 127, 127, 127, 127,...",65,0


In [39]:
train.head()

,Image,Age,Gender
0,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",61,0
1,"[[153, 174, 196, 206, 207, 205, 206, 206, 206,...",40,0
2,"[[253, 253, 253, 252, 252, 252, 251, 252, 252,...",45,0
3,"[[30, 27, 47, 67, 90, 105, 95, 86, 88, 91, 91,...",41,0
4,"[[177, 177, 177, 178, 178, 178, 179, 179, 181,...",54,0


In [40]:
test.head()

,Image,Age,Gender
1471,"[[94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, ...",45,1
2850,"[[145, 145, 145, 145, 145, 145, 145, 145, 145,...",21,0
118,"[[165, 172, 179, 184, 184, 182, 181, 182, 182,...",51,0
729,"[[129, 129, 129, 129, 129, 129, 129, 129, 129,...",26,0
1903,"[[132, 132, 132, 132, 132, 132, 132, 133, 133,...",53,0


In [85]:
x = np.array([[np.array([1,2,3]),[1,2,3]]])
print (x)
x_tf = tf.convert_to_tensor(x)
print (type(x_tf))

[[[1 2 3]
  [1 2 3]]]
<class 'tensorflow.python.framework.ops.EagerTensor'>


In [96]:
def get_batch(x):
  rand_idxs = np.random.choice(train.shape[0], x, True)

  x = train.loc[rand_idxs]["Image"]
  output = np.zeros((x.shape[0],512,512))
  for idx in range((x.shape[0])):
    output[idx]=x[idx]
  #x=np.expand_dims(x, -1)
  print(output)
  print(output.shape)
  for i in x:
    tf.convert_to_tensor(i)
  y = train.loc[rand_idxs]["Gender"]
  print (y)
  y= tf.convert_to_tensor(y)
  print (y)
  print (type(y))
  return x,y
n,m =get_batch(3)
print ("done")
print (n)
print (m)

KeyError: ignored

In [0]:
n_filters = 12 

def classifier(n_outputs=1):
    Conv2D = functools.partial(tf.keras.layers.Conv2D, padding='same', activation='relu')
    BatchNormalization = tf.keras.layers.BatchNormalization
    Flatten = tf.keras.layers.Flatten
    Dense = functools.partial(tf.keras.layers.Dense, activation='relu')

    model = tf.keras.Sequential([
    Conv2D(filters=1*n_filters, kernel_size=5,  strides=2),
    BatchNormalization(),
    
    Conv2D(filters=2*n_filters, kernel_size=5,  strides=2),
    BatchNormalization(),

    Conv2D(filters=4*n_filters, kernel_size=3,  strides=2),
    BatchNormalization(),

    Conv2D(filters=6*n_filters, kernel_size=3,  strides=2),
    BatchNormalization(),

    Flatten(),
    Dense(512),
    Dense(n_outputs, activation=None),
  ])
    return model
My_classifier = classifier()

In [91]:
# Training hyperparameters
batch_size = 32
num_epochs = 2  # keep small to run faster
learning_rate = 5e-4

optimizer = tf.keras.optimizers.Adam(learning_rate) # define our optimizer
loss_history = mdl.util.LossHistory(smoothing_factor=0.99) # to record loss evolution
plotter = mdl.util.PeriodicPlotter(sec=2, scale='semilogy')
if hasattr(tqdm, '_instances'): tqdm._instances.clear() # clear if it exists

@tf.function
def standard_train_step(x, y):
    with tf.GradientTape() as tape:
    # feed the images into the model
        #logits = My_classifier(x)
        print (type(x)) 
  #   # Compute the loss
  #       loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits)

  # # Backpropagation
  #   grads = tape.gradient(loss, My_classifier.trainable_variables)
  #   optimizer.apply_gradients(zip(grads, My_classifier.trainable_variables))
  #   return loss
    return 0

# The training loop!
for epoch in range(num_epochs):
    for idx in tqdm(range(train.shape[0]//batch_size)):
        # Grab a batch of training data and propagate through the network
        x, y = get_batch(batch_size)
        loss = standard_train_step(x, y)

        # # Record the loss and plot the evolution of the loss as a function of training
        # loss_history.append(loss.numpy().mean())
        # plotter.plot(loss_history.get())

  0%|          | 0/101 [00:00<?, ?it/s]

[array([[148, 148, 148, ..., 144, 144, 144],
       [148, 148, 148, ..., 144, 144, 144],
       [148, 148, 148, ..., 144, 144, 144],
       ...,
       [129, 129, 129, ..., 126, 126, 126],
       [129, 129, 129, ..., 126, 126, 126],
       [129, 129, 129, ..., 126, 126, 126]], dtype=uint8)
 array([[105, 104, 108, ...,  26,  26,  26],
       [101,  99, 104, ...,  26,  26,  26],
       [ 91,  88,  96, ...,  25,  25,  25],
       ...,
       [103, 102, 103, ...,   1,   1,   1],
       [101,  99, 103, ...,   2,   1,   1],
       [ 99,  98, 103, ...,   2,   2,   2]], dtype=uint8)
 array([[197, 200, 203, ..., 138, 135, 134],
       [206, 208, 210, ..., 135, 130, 128],
       [219, 220, 220, ..., 129, 122, 118],
       ...,
       [203, 202, 199, ..., 121, 120, 119],
       [197, 196, 194, ..., 127, 126, 125],
       [190, 189, 188, ..., 131, 130, 129]], dtype=uint8)
 array([[160, 159, 143, ..., 152, 166, 166],
       [160, 159, 143, ..., 152, 166, 166],
       [160, 160, 143, ..., 152, 166, 

ValueError: ignored